In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error

In [22]:
df = pd.read_csv("accident.csv")

In [23]:
df.sample(10)

,Age,Gender,Speed_of_Impact,Helmet_Used,Seatbelt_Used,Survived
68,21,Female,44.0,No,No,0
89,33,Male,82.0,No,No,0
167,62,Female,95.0,Yes,Yes,1
101,18,Male,71.0,Yes,Yes,0
73,46,Male,76.0,Yes,No,0
92,64,Male,44.0,Yes,Yes,1
56,45,Male,109.0,Yes,No,0
30,66,Male,72.0,No,Yes,0
12,41,Male,67.0,Yes,No,1
28,61,Male,112.0,Yes,No,1


In [24]:
df.shape

(200, 6)

In [25]:
df.isnull().sum()

Age                0
Gender             0
Speed_of_Impact    3
Helmet_Used        0
Seatbelt_Used      0
Survived           0
dtype: int64

In [26]:
X = df.drop("Survived", axis=1)

In [27]:
X

,Age,Gender,Speed_of_Impact,Helmet_Used,Seatbelt_Used
0,56,Female,27.0,No,No
1,69,Female,46.0,No,Yes
2,46,Male,46.0,Yes,Yes
3,32,Male,117.0,No,Yes
4,60,Female,40.0,Yes,Yes
...,...,...,...,...,...
195,69,Female,111.0,No,Yes
196,30,Female,51.0,No,Yes
197,58,Male,110.0,No,Yes
198,20,Male,103.0,No,Yes


In [28]:
y = df["Survived"]

In [29]:
y

0      1
1      1
2      0
3      0
4      0
      ..
195    1
196    1
197    1
198    1
199    1
Name: Survived, Length: 200, dtype: int64

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.head()

,Age,Gender,Speed_of_Impact,Helmet_Used,Seatbelt_Used
79,53,Male,35.0,Yes,No
197,58,Male,110.0,No,Yes
38,61,Female,106.0,Yes,Yes
24,38,Female,25.0,Yes,Yes
122,24,Male,32.0,No,No


In [32]:
X_train.shape

(160, 5)

In [33]:
y_train

79     0
197    1
38     0
24     0
122    1
      ..
106    1
14     0
92     1
179    0
102    1
Name: Survived, Length: 160, dtype: int64

In [34]:
# imputation transformer
trf1 = ColumnTransformer([
    ('Speed_of_Impact',SimpleImputer(strategy='most_frequent'),[2])
],remainder='passthrough')

In [35]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('Speed_of_Impact',
                                 SimpleImputer(strategy='most_frequent'),
                                 [2])])

In [36]:
# one hot encoding
trf2 = ColumnTransformer([
    ('Helmet_Used', OneHotEncoder(handle_unknown='ignore'), [3]),  
    ('Seatbelt_Used', OneHotEncoder(handle_unknown='ignore'), [4]), 
    ('Gender', OneHotEncoder(handle_unknown='ignore'), [1]) 
], remainder='passthrough')

In [37]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('Helmet_Used',
                                 OneHotEncoder(handle_unknown='ignore'), [3]),
                                ('Seatbelt_Used',
                                 OneHotEncoder(handle_unknown='ignore'), [4]),
                                ('Gender',
                                 OneHotEncoder(handle_unknown='ignore'), [1])])

In [38]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), [0, 2])  # Only scale the numerical columns (Age and Speed_of_Impact)
], remainder='passthrough')

In [39]:
trf3

ColumnTransformer(remainder='passthrough',
                  transformers=[('scale', MinMaxScaler(), [0, 2])])

In [40]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [41]:
trf4

SelectKBest(k=8, score_func=<function chi2 at 0x000001DDA02B74C0>)

In [42]:
# train the model
trf5 = DecisionTreeClassifier()

In [43]:
trf5

DecisionTreeClassifier()

In [44]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

## Pipeline Vs make_pipeline
### Pipeline requires naming of steps, make_pipeline does not.

In [45]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [46]:
# train
pipe.fit(X_train,y_train)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.